<a href="https://colab.research.google.com/github/samkoyun-neo4j/fraud-workshop/blob/main/workshop_notebooks/1-data-ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up & Data ingestion

You could run the workshop from your own local desktop or Google colab.


---
## Context

This notebook allows you load a dataset based on an updated version of [Paysim](https://www.sisu.io/posts/paysim/).  
PaySim is an approximation using an agent-based model and some anonymized, aggregate transactional data from a real mobile money network operator to create synthetic financial data sets academics and hackers can use for exploring ways to detect fraudulent behavior.  
Using this [code](https://github.com/voutilad/paysim), you can generate your own dataset with different caracteristics (size, fraud occurences...).    

We're going to leverage [Neo4j Graph Data Science (GDS)](https://neo4j.com/docs/graph-data-science/current/algorithms/) to investigate through the data and find out fraud patterns and fraudsters.  

## Dataset

The dataset used in this notebook represents money transfers between around 2500 clients, 75 merchants, 5 banks with 175000 transactions across 30 days.  
There are 5 types of transactions:  
* CashIn: a client moves money into the network via a merchant
* CashOut: a client moves money out of the network via a merchant
* Debit: a client moves money into a bank
* Transfer: a client sends money to another client
* Payment: a client exchanges money for something via a merchant

We will try to identify Clients which are fraudsters, potentially targeting other users with fake accounts to accept payments for goods that stolen, illegal or even non existent.  
We added to the original Paysim model some clients details (Phone, Email, SSN) to help identify clients using fake or stolen credentials to gain access to the system (first party fraud).  

---

## Let's get a graph database

We will use a Neo4j graph database created on the [Neo4j sandbox](https://neo4j.com/sandbox/).  
Once connected, on the _Select a project_ page, go to the section _Your own data_ and select the _"Blank Sandbox - Graph Data Science"_.  

Click on the _Create and Download credentials_ button at the bottom of the page.  

<img src="../img/sandbox_select.png" alt="Sandbox Select" width="75%" title="Sandbox Select">  

After few seconds, you should see the below.  
<img src="../img/sandbox_start.png" alt="Sandbox Start" width="75%" title="Sandbox Start">  

And once it's up and running, you can access the connection details by clicking on the top right down arrow and picking the *Connection details* tab.  
You will need 2 things:
* Password  
* Bolt URL   

<img src="../img/sandbox_details.png" alt="Sandbox Details" width="75%" title="Sandbox Details">  

---

## Let's code

First we will import the [Neo4j GDS python library](https://pypi.org/project/graphdatascience/)  

In [16]:
# Install Neo4j GDS Python Client
import sys
!{sys.executable} -m pip install graphdatascience dotenv

# Import our GDS entry point
from graphdatascience import GraphDataScience

### Instantiate your GDS Session

Use Neo4j/Bolt URI and credentials according to your setup  

For local standalone instance Bolt connection without auth    
`gds = GraphDataScience("bolt://localhost:7687", auth=None)`  

For local standalone instance Bolt connection with auth    
`gds = GraphDataScience("bolt://localhost:7687", auth=("neo4j", "<password>"))`  

For remote cluster Neo4j connection with auth  
`gds = GraphDataScience("neo4j://<FQDN or IP Address>:7687", auth=("neo4j", "<password>"))`  

For remote standalone instance Bolt connection with auth   
`gds = GraphDataScience("bolt://<FQDN or IP Address>:7687", auth=("neo4j", "<password>"))`

In [ ]:
from dotenv import load_dotenv
import os

# (Desktop) Load environment variables from a .env file
# load_dotenv(override=True)
# gds = GraphDataScience(os.environ["NEO4J_URI"], auth=(os.environ["NEO4J_USER"], os.environ["NEO4J_PASS"]))

# (Colab) Directly provide connection details (Replace the placeholders below)
gds = GraphDataScience("uri", auth=("neo4j", "password"))

### Check the GDS version installed

In [18]:
print(f"Neo4j GDS Version: {gds.version()}")

Neo4j GDS Version: 2.23.0


### Optional - Set database if you're not using the default _neo4j_ database.

Not applicable for Neo4j Sandbox as we have only one database named _neo4j_.

In [17]:
#gds.set_database("my-db")

### Cleaning the database or making it ready for a rerun of the notebook.
We are starting with a fresh clean database, however if the database was previously loaded, we have the option to clear it out first here. Then we will use it to load the data from CSV files, running [Cypher](https://neo4j.com/developer/cypher/) queries. The RELOAD_DATA flag can be used to skip this step for experimenting with different algorithms later on.

In [19]:
# Set flag to control reloading of all data
RELOAD_DATA = True


if RELOAD_DATA: # Delete all, takes few miniutes on a full database
    gds.run_cypher(
        """
        MATCH (n) CALL {
          WITH n
          DETACH DELETE n
        } IN TRANSACTIONS OF 10 ROWS;
        """
    )
else: # Reset the GDS properties when we re run the book without erasing all
    gds.run_cypher(
        """
        MATCH (c:Client) SET c.fraud_group = null, c.intra_fraud_group = null, c.score = null;
        """
    )

### Test reading some data

Using [LOAD CSV](https://neo4j.com/docs/cypher-manual/current/clauses/load-csv/), we are loading csv files into the database, creating the graph on the fly.  
The first cell is to test the file access, by reading it and showing only the first 5 rows.  

In [20]:
# Checking if we can access the data
if RELOAD_DATA:
    nodeListCSV = gds.run_cypher(
    """
    // LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/samkoyun-neo4j/fraud-workshop/main/data/clients.csv" AS row
    LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/samkoyun-neo4j/fraud-workshop/refs/heads/feature/partner-workshop-edit/data/clients.csv" AS row
    RETURN row.NAME as Name, row.PHONENUMBER as phoneNumber, row.SSN as SSN, row.EMAIL as email 
    """
    )

# The object returned is a Pandas Data Frame, so we can explore using standard Pandas methods
nodeListCSV.head(5)

,Name,phoneNumber,SSN,email
0,Savannah Poole,997-586-014,717-67-2016,poole717@gmail.com
1,Carlos Meyer,547-011-736,494-65-4357,carlosmeyer494@yahoo.com
2,Lauren Moran,570-299-1733,708-40-7191,laurenmoran708@mail.com
3,Genesis Eaton,617-520-484,886-30-2406,genesis.eaton886@mail.com
4,Alexander Fleming,520-481-0734,895-33-9044,alexanderfleming895@mail.com


### Creating constraints and indexes

For data integrity, we will create [constraints](https://neo4j.com/docs/cypher-manual/current/constraints/) to have a robust graph data model. Each constraint enforces uniqueness of an identifier for a given label. An index is also created for the name property on Client nodes, this allows fast lookups when querying clients by name.

In [21]:
if RELOAD_DATA:
    # First we create index
    CONSTRAINTS = [
      "CREATE CONSTRAINT ClientConstraint IF NOT EXISTS FOR (p:Client) REQUIRE p.id IS UNIQUE;",
      "CREATE CONSTRAINT EmailConstraint IF NOT EXISTS FOR (p:Email) REQUIRE p.email IS UNIQUE;",
      "CREATE CONSTRAINT PhoneConstraint IF NOT EXISTS FOR (p:Phone) REQUIRE p.phoneNumber IS UNIQUE;",
      "CREATE CONSTRAINT SSNConstraint IF NOT EXISTS FOR (p:SSN) REQUIRE p.ssn IS UNIQUE;",
      "CREATE CONSTRAINT MerchantConstraint IF NOT EXISTS FOR (p:Merchant) REQUIRE p.id IS UNIQUE;",
      "CREATE CONSTRAINT BankConstraint IF NOT EXISTS FOR (p:Bank) REQUIRE p.id IS UNIQUE;",
      "CREATE CONSTRAINT TransactionConstraint IF NOT EXISTS FOR (p:Transaction) REQUIRE p.globalStep IS UNIQUE;",
      "CREATE CONSTRAINT DebitConstraint IF NOT EXISTS FOR (p:Transaction) REQUIRE p.globalStep IS UNIQUE;",
      "CREATE CONSTRAINT CashInConstraint IF NOT EXISTS FOR (p:CashIn) REQUIRE p.globalStep IS UNIQUE;",
      "CREATE CONSTRAINT CashOutConstraint IF NOT EXISTS FOR (p:CashOut) REQUIRE p.globalStep IS UNIQUE;",
      "CREATE CONSTRAINT TransferConstraint IF NOT EXISTS FOR (p:Transfer) REQUIRE p.globalStep IS UNIQUE;",
      "CREATE CONSTRAINT PaymentConstraint IF NOT EXISTS FOR (p:Payment) REQUIRE p.globalStep IS UNIQUE;",
      "CREATE INDEX      ClientNameIndex IF NOT EXISTS FOR (n:Client) ON (n.name)"
    ]
    for c in CONSTRAINTS:
        gds.run_cypher(c)

### Loading all the data

We will load 7 csv files:  
* one for clients   
* one for merchants  
* five for transactions  

We can see how each node is created with a label and at least one property.  
We see all the relationships between all the nodes, we represent each transaction as a relationship between its participants.  

In [22]:
if RELOAD_DATA:

    # Load Clients data
    gds.run_cypher(
    """
    LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/samkoyun-neo4j/fraud-workshop/refs/heads/feature/partner-workshop-edit/data/clients.csv" AS row
        WITH row
        MERGE (c:Client { id: row.ID })
        SET c.name = row.NAME
        MERGE (p:Phone { phoneNumber: row.PHONENUMBER })
        MERGE (c)-[:HAS_PHONE]->(p)
        MERGE (s:SSN { ssn: row.SSN })
        MERGE (c)-[:HAS_SSN]->(s)
        MERGE (e:Email { email: row.EMAIL })
        MERGE (c)-[:HAS_EMAIL]->(e);
    """
    )

    # Load Merchants data
    gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/samkoyun-neo4j/fraud-workshop/main/data/merchants.csv" AS row
        WITH row
        MERGE (m:Merchant { id: row.ID })
        SET m.name = row.NAME, m.highRisk = toBoolean(row.HIGHRISK);
    """
    )

    # Load Debit data
    gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/samkoyun-neo4j/fraud-workshop/main/data/debit.csv" AS row
        WITH row
        MERGE (b:Bank { id: row.IDDEST })
        SET b.name = row.NAMEDEST
        MERGE (c:Client { id: row.IDORIG })
        MERGE (t:Transaction:Debit { globalStep: toInteger(row.GLOBALSTEP) })
        SET t.amount = toFloat(row.AMOUNT)
        MERGE (t)-[:TO]->(b)
        MERGE (c)-[:PERFORMED]->(t);
    """
    )

    # Load CashIn data, largest file then takes few seconds
    gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/samkoyun-neo4j/fraud-workshop/main/data/cashin.csv" AS row
        CALL {
            WITH row
            MERGE (m:Merchant { id: row.IDDEST })
            SET m.name = row.NAMEDEST
            MERGE (c:Client { id: row.IDORIG })
            MERGE (t:Transaction:CashIn { globalStep: toInteger(row.GLOBALSTEP) })
            SET t.amount = toFloat(row.AMOUNT)
            MERGE (t)-[:TO]->(m)
            MERGE (c)-[:PERFORMED]->(t)
        } IN TRANSACTIONS OF 10 ROWS;
    """
    )

    # Load CashOut data
    gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/samkoyun-neo4j/fraud-workshop/main/data/cashout.csv" AS row
        CALL {
            WITH row
            MERGE (m:Merchant { id: row.IDDEST })
            SET m.name = row.NAMEDEST
            MERGE (c:Client { id: row.IDORIG })
            SET c.name = row.NAMEORIG
            MERGE (t:Transaction:CashOut { globalStep: toInteger(row.GLOBALSTEP) })
            SET t.amount = toFloat(row.AMOUNT)
            MERGE (t)-[:TO]->(m)
            MERGE (c)-[:PERFORMED]->(t)
        } IN TRANSACTIONS OF 10 ROWS;
    """
    )

    # Load Payment data
    gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/samkoyun-neo4j/fraud-workshop/main/data/payment.csv" AS row
        CALL {
            WITH row
            MERGE (m:Merchant { id: row.IDDEST })
            SET m.name = row.NAMEDEST
            MERGE (c:Client { id: row.IDORIG })
            SET c.name = row.NAMEORIG
            MERGE (t:Transaction:Payment { globalStep: toInteger(row.GLOBALSTEP) })
            SET t.amount = toFloat(row.AMOUNT)
            MERGE (t)-[:TO]->(m)
            MERGE (c)-[:PERFORMED]->(t)
        } IN TRANSACTIONS OF 5 ROWS;
    """
    )

    # Load Transfer data
    gds.run_cypher(
    """
    LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/samkoyun-neo4j/fraud-workshop/refs/heads/feature/partner-workshop-edit/data/transfer.csv" AS row
    CALL {
        WITH row
        MERGE (cd:Client { id: row.IDDEST })
        SET cd.name = row.NAMEDEST
        MERGE (co:Client { id: row.IDORIG })
        SET co.name = row.NAMEORIG
        MERGE (t:Transaction:Transfer { globalStep: toInteger(row.GLOBALSTEP) })
        SET t.amount = toFloat(row.AMOUNT)
        MERGE (t)-[:TO]->(cd)
        MERGE (co)-[:PERFORMED]->(t)
    } IN TRANSACTIONS OF 5 ROWS;
    """
    )

We have now taken a series of flat data sources and constructed a rich graph representation of the connections present in the sample dataset. At this point we have the following data model :

<img src="https://github.com/samkoyun-neo4j/fraud-workshop/blob/main/img/initial_data_model.png?raw=1" alt="Initial graph data model" width="75%"  title="Initial Graph Data Model">  

---
### Enriching the graph

Using the transaction details, we are able to enrich the model by adding the ordering of the transaction using the global step (a synthetic timestamp of sorts).

In [23]:
if RELOAD_DATA:
    # Update data model with new relationships
    gds.run_cypher(
    """
    MATCH (c:Client) with c.id as clientId
    CALL {
        WITH clientId
        MATCH (c:Client {id: clientId})-[:PERFORMED]->(tx:Transaction)
        WITH c, tx ORDER BY tx.globalStep
        WITH c, collect(tx) AS txs
        WITH c, txs, head(txs) AS _start, last(txs) AS _last

        MERGE (c)-[:FIRST_TX]->(_start)
        MERGE (c)-[:LAST_TX]->(_last)
        WITH c, apoc.coll.pairsMin(txs) AS pairs

        UNWIND pairs AS pair
          WITH pair[0] AS a, pair[1] AS b
          MERGE (a)-[n:NEXT]->(b)
    } IN TRANSACTIONS OF 10 ROWS;
    """
    )

These changes have created a new layer in the graph, where relationships show transactions in chronological order :

<img src="https://github.com/samkoyun-neo4j/fraud-workshop/blob/main/img/enhanced_data_model.png?raw=1" alt="Enhanced graph data model" width="75%" title="Enhanced Graph Data Model">

This allows us to query and view transaction data in different ways, for example we can show Johns transactions as a group or in their order as below :

Performed transactions | Ordered transactions
-- | --
![alt](https://github.com/samkoyun-neo4j/fraud-workshop/blob/main/img/performed_relationships.png?raw=1) | ![alt](https://github.com/samkoyun-neo4j/fraud-workshop/blob/main/img/ordered_relationships.png?raw=1)
```MATCH path=(:Client {name: "John Kirby"})-[:PERFORMED]->(:Transaction) RETURN path``` | ```MATCH path=(c:Client {name: "John Kirby"})-[:FIRST_TX]->(f_txn)-[:NEXT*]->(l_txn)<-[:LAST_TX]-(c) RETURN path```



### Having a first look at the dataset

Neo4j maintains statistics of the various node labels and relationship types found in the active database. We can gain access to this information using a call to apoc.meta.stats with our python client. Here we are simply asking for the stats and then listing the relative frequency of each label (transaction types, clients, merchants etc) that exist in our dataset.

In [24]:
result = gds.run_cypher(
    """
    CALL apoc.meta.stats() YIELD nodeCount, labels
    UNWIND keys(labels) as label
    RETURN label as nodeLabel,
        labels[label] as frequency,
        round(toFloat(labels[label])/nodeCount, 3) as relativeFrequency
    ORDER BY frequency DESC
    """
)
result

,nodeLabel,frequency,relativeFrequency
0,Transaction,155632,0.972
1,CashIn,70553,0.441
2,CashOut,37469,0.234
3,Payment,35746,0.223
4,Transfer,9626,0.060
5,Debit,2238,0.014
6,Client,1151,0.007
7,SSN,1073,0.007
8,Email,1068,0.007
9,Phone,1068,0.007


### Let's have a look on how the money is exchanged across entities

Here we are using some Cypher aggregations to perform analysis across all transactions in the database. The first phase calculates the total count and value of all transactions. Next we aggregate on each transaction type (label) to calculate the relative value and count percentages of each.

In [25]:
result = gds.run_cypher(
    """
    MATCH (t:Transaction)
    WITH sum(t.amount) AS globalSum, count(t) AS globalCnt
    MATCH (t:Transaction)
    WITH labels(t)[1] as txType, count(t) as txCnt, sum(t.amount) as txTotal, globalSum, globalCnt
    RETURN
        txType,
        toInteger(round(txTotal/1000000)) + 'M' AS TotalMarketValue,
        round(100 * txTotal / globalSum, 1) AS `%MarketValue`,
        round(100 * toFloat(txCnt) / globalCnt, 1) AS `%MarketTransactions`,
        toInteger(txTotal / txCnt) AS AvgTransactionValue,
        txCnt AS NumberOfTransactions
    ORDER BY `%MarketTransactions` DESC
    """
)
result

,txType,TotalMarketValue,%MarketValue,%MarketTransactions,AvgTransactionValue,NumberOfTransactions
0,CashIn,9748M,42.3,45.3,138165,70553
1,CashOut,4753M,20.6,24.1,126846,37469
2,Payment,312M,1.4,23.0,8736,35746
3,Transfer,8219M,35.7,6.2,853865,9626
4,Debit,13M,0.1,1.4,5847,2238
